In [8]:
import numpy as np

def softmax(ratings):
    """ Compute the softmax of a list of numbers w. """
    e = np.exp(np.array(ratings))
    dist = e / np.sum(e)
    return dist

def update_elo(winner_elo, loser_elo):
    """ Update ELO ratings after a game. """
    winner_update = 0.1 * loser_elo
    loser_update = 0.1 * (10 - winner_elo)
    new_winner_elo = min(10, winner_elo + winner_update)
    new_loser_elo = max(0, loser_elo - loser_update)
    return new_winner_elo, new_loser_elo

def simulate_season(initial_ratings, hot=True):
    """ Simulate a season with given initial ratings. """
    ratings = initial_ratings.copy()
    wins = [0] * len(ratings)

    for _ in range(100):  # 100 games in a season
        # Randomly select two different teams
        teams = np.random.choice(len(ratings), 2, replace=False)
        win_prob = softmax([ratings[teams[0]], ratings[teams[1]]])

        # Determine the winner
        winner = teams[0] if np.random.rand() < win_prob[0] else teams[1]
        loser = teams[1] if winner == teams[0] else teams[0]

        # Update win count
        wins[winner] += 1

        # Update ratings if it's a hot simulation
        if hot:
            ratings[winner], ratings[loser] = update_elo(ratings[winner], ratings[loser])

        teams2 = [i for i in range(len(ratings)) if i not in teams]
        win_prob2 = softmax([ratings[teams2[0]], ratings[teams2[1]]])

        # Determine the winner
        winner2 = teams2[0] if np.random.rand() < win_prob2[0] else teams2[1]
        loser2 = teams2[1] if winner2 == teams2[0] else teams2[0]

        # Update win count
        wins[winner2] += 1

        # Update ratings if it's a hot simulation
        if hot:
            ratings[winner2], ratings[loser2] = update_elo(ratings[winner2], ratings[loser2])
    

    return wins

def run_simulations(num_simulations=100):
    """ Run multiple simulations and compare static vs hot simulations. """
    initial_ratings = [2, 4, 6, 8]  # Initial ratings of the teams
    static_wins = np.zeros((num_simulations, len(initial_ratings)))
    hot_wins = np.zeros((num_simulations, len(initial_ratings)))

    for i in range(num_simulations):
        static_wins[i] = simulate_season(initial_ratings, hot=False)
        hot_wins[i] = simulate_season(initial_ratings, hot=True)

    return static_wins, hot_wins

# Run the simulations
num_simulations = 100
static_wins, hot_wins = run_simulations(num_simulations=num_simulations)



In [9]:
static_wins.mean(axis=0), hot_wins.mean(axis=0)

(array([ 4.8 , 33.45, 66.5 , 95.25]), array([13.2 , 24.2 , 73.73, 88.87]))

In [10]:
static_wins.std(axis=0), hot_wins.std(axis=0)

(array([1.92353841, 4.72731425, 4.70212718, 2.333988  ]),
 array([ 7.51797845, 12.12682976, 14.78367681,  5.87648705]))